<a href="https://colab.research.google.com/github/dinesh-umkc/kdm/blob/main/ICP6_LCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LCA- Topic Modeling
#Objective
1. Get books from https://www.gutenberg.org 
2. Clean content from book data
3. Get topics using LCA
4. Get content from these threads
 - 2meirl4meirl — 842,000 subscribers
 - disneyvacation — 478,000 subscribers
 - unresolvedmysteries — 732,000 subscribers
 - wewantplates — 502,000 subscribers
 - antiMLM — 493,000 subscribers
5. Clean content from thread data
6. Get topics using LCA

In [16]:
import urllib
from bs4 import BeautifulSoup

from urllib.request import urlopen
from bs4 import BeautifulSoup
documents = []

import re
regex = re.compile(r'<[^>]+>')

def remove_html(string):
    return regex.sub('', string)

def getContent(url): 
  #url = "https://www.gutenberg.org/files/55/55-h/55-h.htm" 
  html = urlopen(url).read()
  soup = BeautifulSoup(html, features="html.parser")
  for script in soup(["script", "style"]):
    script.extract()
  text = soup.get_text()
  documents.append(remove_html(text))

url="https://www.gutenberg.org/files/55/55-h/55-h.htm"
getContent(url)
(documents[0])[:500]
url = "https://www.gutenberg.org/files/54/54-h/54-h.htm" 
getContent(url)
url = "https://www.gutenberg.org/files/33361/33361-h/33361-h.htm" 
getContent(url)
url = "https://www.gutenberg.org/files/22566/22566-h/22566-h.htm" 
getContent(url)
url = "https://www.gutenberg.org/files/26624/26624-h/26624-h.htm" 
getContent(url)



In [45]:
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

tfidf_vect = CountVectorizer(analyzer='word', stop_words = 'english')
df=tfidf_vect.fit_transform(documents)
vocab = tfidf_vect.get_feature_names_out()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
vocab

array(['00', '000', '10', ..., 'zoroaster', 'zuz', 'zy'], dtype=object)

In [40]:
print(df.shape)

(5, 8851)


In [41]:
print(df[0])

  (0, 5177)	0.0010695255139764681
  (0, 7601)	0.0010695255139764681
  (0, 5952)	0.0010695255139764681
  (0, 2958)	0.0010695255139764681
  (0, 5626)	0.0010695255139764681
  (0, 4749)	0.0010695255139764681
  (0, 2655)	0.0010695255139764681
  (0, 5136)	0.0010695255139764681
  (0, 1783)	0.0010695255139764681
  (0, 8434)	0.0010695255139764681
  (0, 5168)	0.0010695255139764681
  (0, 4675)	0.0010695255139764681
  (0, 5953)	0.0010695255139764681
  (0, 6889)	0.0010695255139764681
  (0, 5361)	0.0010695255139764681
  (0, 3775)	0.0010695255139764681
  (0, 4913)	0.0010695255139764681
  (0, 5957)	0.0010695255139764681
  (0, 1352)	0.0010695255139764681
  (0, 2005)	0.0010695255139764681
  (0, 8527)	0.0010695255139764681
  (0, 401)	0.0010695255139764681
  (0, 4910)	0.0010695255139764681
  (0, 2484)	0.0010695255139764681
  (0, 5427)	0.0010695255139764681
  :	:
  (0, 4644)	0.007486678597835278
  (0, 5353)	0.010695255139764681
  (0, 8794)	0.010695255139764681
  (0, 5320)	0.004278102055905873
  (0, 4108)	0

In [43]:
print (vocab[5177])

newsletter


In [46]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components = 5, doc_topic_prior=1)
lda.fit(df)
lda.components_[0].shape

(8851,)

In [48]:
import numpy as np 
topic_words = {}
n_top_words = 20
for topic, comp in enumerate(lda.components_):
    # print(topic, comp)
    word_idx = np.argsort(comp)[::-1][:n_top_words] #argsort to get index, and [::-1] to sort in descending
    # store the words most relevant to the topic
    topic_words[topic] = [vocab[i] for i in word_idx]
    # break
    
for topic, words in topic_words.items():
    print('Topic: %d' % topic)
    print('  %s' % ', '.join(words))

Topic: 0
  dorothy, said, scarecrow, woodman, lion, oz, great, tin, little, witch, asked, green, came, good, gutenberg, girl, toto, head, project, shall
Topic: 1
  said, tip, scarecrow, horse, jack, saw, woodman, tin, pumpkinhead, mombi, city, boy, woggle, bug, head, gutenberg, glinda, good, old, project
Topic: 2
  actual, shown, tinkling, accepting, spoil, boards, furniture, towers, stern, missing, errors, assorted, putting, throwing, preserve, dull, strict, heartily, secure, main
Topic: 3
  dorothy, said, man, little, wizard, shaggy, pg, asked, bright, gutenberg, ozma, button, project, oz, like, don, time, good, jim, people
Topic: 4
  dorothy, said, pg, king, billina, ozma, hen, scarecrow, girl, nome, little, tiktok, ev, tin, oz, gutenberg, palace, project, asked, woodman
